# Even all the cleaned articles are about rel_words, AI. Not every sentence is about AI! Some non-AI sentences are negative.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
import ast
from sklearn.preprocessing import StandardScaler
import wordcloud
import re
import os

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

import warnings
warnings.simplefilter('ignore')

In [3]:
from google.colab import drive
import json

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the file path
file_path = '/content/drive/My Drive/Colab Notebooks/NLP_Ignas/nlp_finalproject/'

In [5]:
%%time
# Each row is an article
df_cleaned_final = pd.read_csv(os.path.join(file_path, 'df_cleaned_final.csv'))
# Display the DataFrame
df_cleaned_final.head(2)

CPU times: user 11.5 s, sys: 1.78 s, total: 13.3 s
Wall time: 19.1 s


,article_id,url,date,language,title,text,token_count
0,1,http://spaceref.com/astronomy/observation-simulation-and-ai-join-forces-to-reveal-a-clear-universe.html,2021-07-05,en,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,912
1,2,http://www.agoravox.it/Covid-19-un-messaggio-dai.html,2020-03-13,en,Covid-19 un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia,Covid-19 un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia Accedi Partecipa ad AgoraVox Iscriviti e proponi un articolo Home page Attualit Ambiente Cronaca Cronaca Locale Cultura Economia Europa Media Istruzione Mondo Politica Salute Religione Societ Scienza e Tecnologia Tribuna Libera Di' la tua Tempo Libero Gossip Redazionali Concorsi Cinema FameTulipani Incredibile ma vero La vignetta del giorno Moda e tendenze Pubblicazioni AgoraVox Recensioni Satira Musica e ...,2039


In [6]:
df_cleaned_final.shape

(153749, 7)

In [7]:
%%time
# Each row is an article
df_topic_modeling = pd.read_csv(os.path.join(file_path, 'topic_modeling.csv'))
# Display the DataFrame
df_topic_modeling.head(2)

CPU times: user 28.6 s, sys: 3.51 s, total: 32.1 s
Wall time: 1min 6s


,article_id,url,date,language,title,text,token_count,text_cleaned,title_cleaned,title_text_cleaned,topic
0,1,http://spaceref.com/astronomy/observation-simulation-and-ai-join-forces-to-reveal-a-clear-universe.html,2021-07-05,en,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,912,observation simulation ai join force reveal clear universe spaceref home nasa watch spaceref business astrobiology web advertising add event sign daily newsletter international space station nasa hack space calendar mission space weather observation simulation ai join force reveal clear universe press release source national institute natural science posted july pm view comment using ai driven data analysis peel back noise find actual shape universe credit institute statistical mathematics j...,observation simulation ai join force reveal clear universe spaceref,observation simulation ai join force reveal clear universe spaceref home nasa watch spaceref business astrobiology web advertising add event sign daily newsletter international space station nasa hack space calendar mission space weather observation simulation ai join force reveal clear universe press release source national institute natural science posted july pm view comment using ai driven data analysis peel back noise find actual shape universe credit institute statistical mathematics j...,AIResearch
1,2,http://www.agoravox.it/Covid-19-un-messaggio-dai.html,2020-03-13,en,Covid-19 un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia,Covid-19 un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia Accedi Partecipa ad AgoraVox Iscriviti e proponi un articolo Home page Attualit Ambiente Cronaca Cronaca Locale Cultura Economia Europa Media Istruzione Mondo Politica Salute Religione Societ Scienza e Tecnologia Tribuna Libera Di' la tua Tempo Libero Gossip Redazionali Concorsi Cinema FameTulipani Incredibile ma vero La vignetta del giorno Moda e tendenze Pubblicazioni AgoraVox Recensioni Satira Musica e ...,2039,covid un messaggio dai ricercatori italiani ai colleghi stranieri agoravox italia accedi partecipa ad agoravox iscriviti e proponi un articolo home page attualit ambiente cronaca cronaca locale cultura economia europa medium istruzione mondo politica salute religione societ scienza e tecnologia tribuna libera di la tua tempo libero gossip redazionali concorsi cinema fametulipani incredibile vero la vignetta del giorno moda e tendenze pubblicazioni agoravox recensioni satira musica e spettaco...,covid un messaggio dai ricercatori italiani ai colleghi stranieri agoravox italia,covid un messaggio dai ricercatori italiani ai colleghi stranieri agoravox italia accedi partecipa ad agoravox iscriviti e proponi un articolo home page attualit ambiente cronaca cronaca locale cultura economia europa medium istruzione mondo politica salute religione societ scienza e tecnologia tribuna libera di la tua tempo libero gossip redazionali concorsi cinema fametulipani incredibile vero la vignetta del giorno moda e tendenze pubblicazioni agoravox recensioni satira musica e spettaco...,HealthcareTech


In [8]:
df_topic_modeling.shape

(153749, 11)

In [9]:
%%time
# Each row is a sentence
df_sentences = pd.read_csv(os.path.join(file_path, 'sentences.csv'))
# Display the DataFrame
df_sentences.head(2)

CPU times: user 14.2 s, sys: 1.36 s, total: 15.6 s
Wall time: 26.7 s


,article_id,sentence_id,sentence
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.


In [10]:
df_sentences.shape

(6523835, 3)

In [11]:
%%time
# Each row is a sentence
df_sentences_with_entities = pd.read_csv(os.path.join(file_path, 'sentences_with_entities.csv'))
# Display the DataFrame
df_sentences_with_entities.head(2)

CPU times: user 19.4 s, sys: 1.74 s, total: 21.1 s
Wall time: 39.1 s


,article_id,sentence_id,sentence,entities
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,"[('July 4 2021', 'DATE'), ('AI', 'PRODUCT'), ('Universe', 'ORG')]"
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,"[('CREDIT The Institute of Statistical Mathematics Japanese', 'ORG'), ('AI', 'PRODUCT')]"


In [12]:
df_sentences_with_entities.shape

(6523835, 4)

In [13]:
%%time
# Each row is a sentence
df_sentences_with_job_ind_tech = pd.read_csv(os.path.join(file_path, 'sentences_with_job_ind_tech.csv'))
# Display the DataFrame
df_sentences_with_job_ind_tech.head(2)

CPU times: user 14.6 s, sys: 1.32 s, total: 15.9 s
Wall time: 28.6 s


,article_id,sentence_id,sentence,entities_combined
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,[]
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,[]


In [14]:
df_sentences_with_job_ind_tech.shape

(6523835, 4)

In [15]:
%%time
# Each row is a sentence
df_sentences_with_sentiments_certainty = pd.read_csv(os.path.join(file_path, 'sentences_with_sentiments_certainty.csv'))
# Display the DataFrame
df_sentences_with_sentiments_certainty.head(2)

CPU times: user 17.4 s, sys: 1.81 s, total: 19.2 s
Wall time: 26.3 s


,article_id,sentence_id,sentence,__index_level_0__,sentiment_score_DistilBERT,sentiment_label_DistilBERT,sentiment_label_certainty
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,0,0.989424,NEUTRAL,0.000000
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,1,0.541715,POSITIVE,0.541715


In [16]:
df_sentences_with_sentiments_certainty.shape

(6523802, 7)

### Merge the dataframes on sentence level

In [17]:
# Perform the left outer merge and include only the 'entities' column from df_sentences_with_entities
df_merged = df_sentences.merge(
    df_sentences_with_entities[['article_id', 'sentence_id', 'entities']],  # Select only relevant columns
    on=['article_id', 'sentence_id'],
    how='left'
)
df_merged.head()

,article_id,sentence_id,sentence,entities
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,"[('July 4 2021', 'DATE'), ('AI', 'PRODUCT'), ('Universe', 'ORG')]"
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,"[('CREDIT The Institute of Statistical Mathematics Japanese', 'ORG'), ('AI', 'PRODUCT')]"
2,1,3,After extensive training and testing on large mock data created by supercomputer simulations they then applied this new tool to actual data from Japan's Subaru Telescope and found that the mass distribution derived from using this method is consistent with the currently accepted models of the Universe.,"[('Japan', 'GPE'), ('Subaru Telescope', 'PRODUCT'), ('Universe', 'ORG')]"
3,1,4,This is a powerful new tool for analyzing big data from current and planned astronomy surveys.,[]
4,1,5,Wide area survey data can be used to study the large-scale structure of the Universe through measurements of gravitational lensing patterns.,"[('Universe', 'ORG')]"


In [18]:
df_merged = df_merged.merge(
    df_sentences_with_job_ind_tech[['article_id', 'sentence_id', 'entities_combined']],  # Select only relevant columns
    on=['article_id', 'sentence_id'],
    how='left'
)
df_merged = df_merged[['article_id', 'sentence_id', 'sentence', 'entities', 'entities_combined']]
df_merged.head()

,article_id,sentence_id,sentence,entities,entities_combined
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,"[('July 4 2021', 'DATE'), ('AI', 'PRODUCT'), ('Universe', 'ORG')]",[]
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,"[('CREDIT The Institute of Statistical Mathematics Japanese', 'ORG'), ('AI', 'PRODUCT')]",[]
2,1,3,After extensive training and testing on large mock data created by supercomputer simulations they then applied this new tool to actual data from Japan's Subaru Telescope and found that the mass distribution derived from using this method is consistent with the currently accepted models of the Universe.,"[('Japan', 'GPE'), ('Subaru Telescope', 'PRODUCT'), ('Universe', 'ORG')]",[]
3,1,4,This is a powerful new tool for analyzing big data from current and planned astronomy surveys.,[],"[['Big Data', 'TECH']]"
4,1,5,Wide area survey data can be used to study the large-scale structure of the Universe through measurements of gravitational lensing patterns.,"[('Universe', 'ORG')]",[]


In [19]:
df_merged.shape

(6523835, 5)

In [20]:
df_sentences_with_sentiments_certainty.shape

(6523802, 7)

In [21]:
df_merged = df_merged.merge(
    df_sentences_with_sentiments_certainty[['article_id', 'sentence_id', 'sentiment_score_DistilBERT', 'sentiment_label_DistilBERT', 'sentiment_label_certainty']],  # Select only relevant columns
    on=['article_id', 'sentence_id'],
    how='inner'
)
df_merged = df_merged[['article_id', 'sentence_id', 'sentence', 'entities', 'entities_combined', 'sentiment_score_DistilBERT', 'sentiment_label_DistilBERT', 'sentiment_label_certainty']]
df_merged.head()

,article_id,sentence_id,sentence,entities,entities_combined,sentiment_score_DistilBERT,sentiment_label_DistilBERT,sentiment_label_certainty
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,"[('July 4 2021', 'DATE'), ('AI', 'PRODUCT'), ('Universe', 'ORG')]",[],0.989424,NEUTRAL,0.000000
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,"[('CREDIT The Institute of Statistical Mathematics Japanese', 'ORG'), ('AI', 'PRODUCT')]",[],0.541715,POSITIVE,0.541715
2,1,3,After extensive training and testing on large mock data created by supercomputer simulations they then applied this new tool to actual data from Japan's Subaru Telescope and found that the mass distribution derived from using this method is consistent with the currently accepted models of the Universe.,"[('Japan', 'GPE'), ('Subaru Telescope', 'PRODUCT'), ('Universe', 'ORG')]",[],0.933930,NEUTRAL,0.000000
3,1,4,This is a powerful new tool for analyzing big data from current and planned astronomy surveys.,[],"[['Big Data', 'TECH']]",0.789955,NEUTRAL,0.000000
4,1,5,Wide area survey data can be used to study the large-scale structure of the Universe through measurements of gravitational lensing patterns.,"[('Universe', 'ORG')]",[],0.990115,NEUTRAL,0.000000


In [22]:
df_merged.shape

(6523802, 8)

### Merge article level info into sentence level dataframe

In [23]:
df_topic_modeling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153749 entries, 0 to 153748
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   article_id          153749 non-null  int64 
 1   url                 153749 non-null  object
 2   date                153749 non-null  object
 3   language            153749 non-null  object
 4   title               153749 non-null  object
 5   text                153749 non-null  object
 6   token_count         153749 non-null  int64 
 7   text_cleaned        153749 non-null  object
 8   title_cleaned       153749 non-null  object
 9   title_text_cleaned  153749 non-null  object
 10  topic               153749 non-null  object
dtypes: int64(2), object(9)
memory usage: 12.9+ MB


In [24]:
df_merged = df_merged.merge(
    df_topic_modeling[['article_id', 'date', 'topic']],  # Select only relevant columns
    on=['article_id'],
    how='left'
)
df_merged = df_merged[['article_id', 'sentence_id', 'sentence', 'entities', 'entities_combined', 'sentiment_score_DistilBERT', 'sentiment_label_DistilBERT', 'sentiment_label_certainty', 'date', 'topic']]
df_merged.head()

,article_id,sentence_id,sentence,entities,entities_combined,sentiment_score_DistilBERT,sentiment_label_DistilBERT,sentiment_label_certainty,date,topic
0,1,1,Observation Simulation And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA Watch SpaceRef Business Astrobiology Web Advertising Add an Event Sign up for our Daily Newsletter International Space Station NASA Hack Space Calendar Missions Space Weather Observation Simulation And AI Join Forces To Reveal A Clear Universe Press Release - Source NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4 2021 1000 PM View Comments Using AI driven data analysis to peel back the noise and f...,"[('July 4 2021', 'DATE'), ('AI', 'PRODUCT'), ('Universe', 'ORG')]",[],0.989424,NEUTRAL,0.000000,2021-07-05,AIResearch
1,1,2,CREDIT The Institute of Statistical Mathematics Japanese astronomers have developed a new artificial intelligence AI technique to remove noise in astronomical data due to random variations in galaxy shapes.,"[('CREDIT The Institute of Statistical Mathematics Japanese', 'ORG'), ('AI', 'PRODUCT')]",[],0.541715,POSITIVE,0.541715,2021-07-05,AIResearch
2,1,3,After extensive training and testing on large mock data created by supercomputer simulations they then applied this new tool to actual data from Japan's Subaru Telescope and found that the mass distribution derived from using this method is consistent with the currently accepted models of the Universe.,"[('Japan', 'GPE'), ('Subaru Telescope', 'PRODUCT'), ('Universe', 'ORG')]",[],0.933930,NEUTRAL,0.000000,2021-07-05,AIResearch
3,1,4,This is a powerful new tool for analyzing big data from current and planned astronomy surveys.,[],"[['Big Data', 'TECH']]",0.789955,NEUTRAL,0.000000,2021-07-05,AIResearch
4,1,5,Wide area survey data can be used to study the large-scale structure of the Universe through measurements of gravitational lensing patterns.,"[('Universe', 'ORG')]",[],0.990115,NEUTRAL,0.000000,2021-07-05,AIResearch


In [25]:
df_merged.shape

(6523802, 10)

In [26]:
df_merged.topic.value_counts()

,count
topic,
other,2349898
MarketResearch,754419
EntertainmentTech,730237
PolicyandLegal,575132
CloudComputing,441876
Finance,409492
TechProducts,264927
AIinContentCreation,232287
Media,227870


In [27]:
# # Exclude topic = 'other'
# df_merged = df_merged[df_merged['topic']!='other']

In [28]:
df_merged.shape

(6523802, 10)

In [29]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6523802 entries, 0 to 6523801
Data columns (total 10 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   article_id                  int64  
 1   sentence_id                 int64  
 2   sentence                    object 
 3   entities                    object 
 4   entities_combined           object 
 5   sentiment_score_DistilBERT  float64
 6   sentiment_label_DistilBERT  object 
 7   sentiment_label_certainty   float64
 8   date                        object 
 9   topic                       object 
dtypes: float64(2), int64(2), object(6)
memory usage: 497.7+ MB


In [30]:
# Save the DataFrame as a CSV file
df_merged.to_csv(file_path + 'df_merged.csv', index=False)